In [32]:
Sarcasm_Path = '/sarcasm_headlines.csv'

In [33]:
import pandas as pd

def read_and_view_csv(Sarcasm_Path):
    sarcasm_headlines = pd.read_csv(Sarcasm_Path)
    sarcasm_headlines.dropna(inplace=True)
    print("Shape= ", sarcasm_headlines.shape)

    print("Columns names:", sarcasm_headlines.columns.tolist())
    return sarcasm_headlines
sarcasm_headlines= read_and_view_csv(Sarcasm_Path)

Shape=  (26600, 3)
Columns names: ['headline', 'is_sarcastic', 'Cleaned_Headlines']


In [34]:
sarcasm_headlines = sarcasm_headlines[['is_sarcastic','Cleaned_Headlines']] #take cleaned headlines content and the output variables.

In [35]:
sarcasm_headlines

,is_sarcastic,Cleaned_Headlines
0,0,former versace store clerk sues secret black c...
1,0,roseanne revival catches thorny political mood...
2,1,mom starting fear son web series closest thing...
3,1,boehner wants wife listen come alternative deb...
4,0,j k rowling wishes snape happy birthday magica...
...,...,...
26597,0,american politics moral free fall
26598,0,america best hikes
26599,0,reparations obama
26600,0,israeli ban targeting boycott supporters raise...


In [36]:
import warnings as WARN
WARN.filterwarnings('ignore')

In [37]:
sarcasm_headlines['is_sarcastic'].value_counts()

,count
is_sarcastic,
0,14949
1,11651


In [38]:
X = sarcasm_headlines['Cleaned_Headlines']
y = sarcasm_headlines['is_sarcastic']

In [39]:
!pip install transformers

In [40]:
import torch

from transformers import BertTokenizer
from transformers import BertModel

In [41]:
Tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
Bert_Model = BertModel.from_pretrained('bert-base-uncased')

Bert_Model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [42]:
# Tokenize and encode the text content

def Encoder(headlines):
    headlines = headlines.tolist()
    Encoder = Tokenizer(headlines, truncation=True, padding=True, max_length=10, return_tensors='pt')
    with torch.no_grad():
        Embedder = Bert_Model(**Encoder).last_hidden_state
    return Embedder.mean(dim=1).numpy()

In [45]:
X = Encoder(X)
print(X)

[[ 0.37828785 -0.2973457   0.49049592 ... -0.28869873 -0.08718195
  -0.02915508]
 [-0.41077477 -0.1556618   0.34250608 ... -0.3515918  -0.22724858
  -0.17816813]
 [ 0.2543492  -0.01970676  0.31810132 ... -0.14033629  0.18797606
  -0.22335169]
 ...
 [-0.2724633   0.00156015 -0.09744564 ...  0.21189456 -0.16519496
  -0.06639819]
 [ 0.28311902 -0.02907328 -0.37016708 ... -0.447323   -0.38668942
  -0.18544394]
 [ 0.07674705 -0.16003513  0.1233006  ... -0.06436368 -0.07252679
  -0.41399089]]


In [51]:
X.shape

(26600, 768)

In [52]:
from sklearn.decomposition import PCA

decomposer_PCA = PCA(n_components = 10)
X = decomposer_PCA.fit_transform(X)
X.shape

(26600, 10)

In [77]:
from sklearn.model_selection import train_test_split as split

def train_test(X, y):

    X_trn, X_tst, y_trn, y_tst = split(X, y, test_size=0.4, random_state=1)
    X_val, X_tst, y_val, y_tst = split(X_tst, y_tst, test_size=0.5, random_state=1)

    return X_trn, X_tst, X_val, y_trn, y_tst, y_val

X_trn, X_tst, X_val, y_trn, y_tst, y_val = train_test(X,y)

print("Training data size:", len(X_trn))
print("Testing data size:", len(X_tst))
print("validation data size:", len(X_val))


Training data size: 15960
Testing data size: 5320
validation data size: 5320


In [78]:
X_trn

array([[ 0.9170738 ,  0.60776126,  0.39818373, ..., -0.18089885,
         0.21055278,  0.79859513],
       [ 0.9687702 ,  0.3239609 ,  2.2418523 , ..., -2.5869687 ,
         0.25157604, -1.2571727 ],
       [ 1.6854246 , -0.36606884, -0.27546173, ...,  0.2230162 ,
        -0.70854855,  0.25048915],
       ...,
       [ 0.11106175, -1.2487441 ,  0.16450405, ...,  0.19515267,
        -0.22665161,  0.6371676 ],
       [ 1.7337141 , -0.43770635,  0.56862956, ...,  1.1896708 ,
         0.16667223, -0.03048968],
       [-2.908323  ,  0.21414733, -1.4884351 , ...,  0.3956958 ,
        -0.39096862, -0.28426573]], dtype=float32)

In [79]:
X_tst

array([[ 2.624517  ,  2.3722868 ,  1.6882867 , ..., -0.99221873,
        -0.07274921,  0.36435184],
       [ 0.60179144,  0.26901937, -0.7446264 , ..., -0.10124417,
        -1.1294309 ,  0.05013534],
       [-0.8611665 , -2.328494  ,  0.00584003, ..., -0.29996616,
         0.8045643 ,  1.0349526 ],
       ...,
       [-0.45144194,  0.6058713 , -1.0108489 , ..., -0.2252883 ,
        -0.78463906, -1.2607468 ],
       [-0.64220476,  0.5064155 ,  0.03501225, ..., -0.1949762 ,
        -0.20499355,  0.51027095],
       [-1.3037832 ,  0.551676  , -0.43794703, ...,  1.979774  ,
         1.1135439 ,  0.3525592 ]], dtype=float32)

In [80]:
X_val

array([[-1.0619718 ,  1.497128  ,  0.32646158, ..., -0.25151247,
        -0.5182971 , -0.35976088],
       [ 1.2553389 ,  0.59272695,  0.24937925, ...,  1.0756662 ,
        -0.6484829 , -0.62361825],
       [-0.7074942 , -0.41064525, -1.0658947 , ...,  1.0220182 ,
         0.15749937,  0.9667665 ],
       ...,
       [ 1.8883061 , -0.5257155 ,  0.2428596 , ...,  0.5350341 ,
         1.1020101 ,  1.0210328 ],
       [-0.7670059 ,  0.66115016,  0.5652051 , ..., -0.0881114 ,
         0.66319954,  0.2474241 ],
       [ 3.098075  , -0.8536898 ,  0.6475509 , ...,  0.4407609 ,
        -0.73976755,  0.39440566]], dtype=float32)

**Machine learning models (SVM, Random Forest, Logistic Regression)**

In [65]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import GridSearchCV as Grid
from sklearn.metrics import accuracy_score as Score
from sklearn.metrics import classification_report as Report

SVM

In [63]:
grid_parameter= {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

svm_mod = SVC()

grid_var = Grid(svm_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
svm_mod = grid_var.best_estimator_

Y_pred = svm_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
Best Parameters: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}


, Validation Score: 0.7137
              precision    recall  f1-score   support

           0       0.72      0.80      0.76      3015
           1       0.70      0.60      0.64      2305

    accuracy                           0.71      5320
   macro avg       0.71      0.70      0.70      5320
weighted avg       0.71      0.71      0.71      5320

Macro_F1_score 0.7023694665064633


In [64]:
Y_pred = svm_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.7233
              precision    recall  f1-score   support

           0       0.73      0.80      0.76      2986
           1       0.71      0.63      0.66      2334

    accuracy                           0.72      5320
   macro avg       0.72      0.71      0.71      5320
weighted avg       0.72      0.72      0.72      5320

Macro_F1_score 0.714663447419275


Random Forest

In [66]:
grid_parameter= {
    'n_estimators': [50, 10, 20],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

RF_mod = RF()

grid_var = Grid(RF_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
RF_mod = grid_var.best_estimator_

Y_pred = RF_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)


Fitting 2 folds for each of 36 candidates, totalling 72 fits
Best Parameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50}


, Validation Score: 0.7070
              precision    recall  f1-score   support

           0       0.72      0.79      0.75      3015
           1       0.68      0.60      0.64      2305

    accuracy                           0.71      5320
   macro avg       0.70      0.69      0.70      5320
weighted avg       0.71      0.71      0.70      5320

Macro_F1_score 0.6966397889922542


In [67]:
Y_pred = RF_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.7060
              precision    recall  f1-score   support

           0       0.72      0.77      0.75      2986
           1       0.68      0.63      0.65      2334

    accuracy                           0.71      5320
   macro avg       0.70      0.70      0.70      5320
weighted avg       0.70      0.71      0.70      5320

Macro_F1_score 0.6985716016343433


Logistic Regression

In [68]:
grid_parameter= {
    'C': [0.01, 0.1, 1],
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': [10, 20, 30]
}

LR_mod = LR()

grid_var = Grid(LR_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
LR_mod = grid_var.best_estimator_

Y_pred = LR_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)


Fitting 2 folds for each of 27 candidates, totalling 54 fits
Best Parameters: {'C': 0.01, 'max_iter': 10, 'solver': 'liblinear'}


, Validation Score: 0.6863
              precision    recall  f1-score   support

           0       0.70      0.78      0.74      3015
           1       0.66      0.57      0.61      2305

    accuracy                           0.69      5320
   macro avg       0.68      0.67      0.67      5320
weighted avg       0.68      0.69      0.68      5320

Macro_F1_score 0.673787734095789


In [69]:
Y_pred = LR_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.6876
              precision    recall  f1-score   support

           0       0.71      0.76      0.73      2986
           1       0.66      0.60      0.63      2334

    accuracy                           0.69      5320
   macro avg       0.68      0.68      0.68      5320
weighted avg       0.69      0.69      0.69      5320

Macro_F1_score 0.6792111106273673


**Deep learning models (LSTM, BiLSTM)**

In [72]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
import numpy as np

LSTM

In [81]:
# hyper tuning the optimizer
for optimizer in ['adam', 'adamax', 'adagrad']:
  print("Result of hyper tuning the optimizer ", optimizer)

  LSTM_mod = Sequential()
  LSTM_mod.add(LSTM(200, input_shape=(X_trn.shape[1], 1)))
  LSTM_mod.add(Dense(1, activation='sigmoid'))
  LSTM_mod.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  LSTM_mod.fit(X_trn, y_trn, epochs=20, batch_size=128)

  Y_pred = LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the optimizer  adam
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 76ms/step - accuracy: 0.5534 - loss: 0.6821
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - accuracy: 0.5905 - loss: 0.6615
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 84ms/step - accuracy: 0.6214 - loss: 0.6425
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.6656 - loss: 0.6101
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 106ms/step - accuracy: 0.6852 - loss: 0.5915
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 106ms/step - accuracy: 0.6889 - loss: 0.5888
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.6944 - loss: 0.5822
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step - accuracy: 0.7020 - loss: 0.5710
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.6943 - loss: 0.5755
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.7046 - loss: 0.5672
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.7029 

In [82]:
# hyper tuning the count of epoches
for Epoches in [5, 10, 20]:
  print("Result of hyper tuning the count of epoches ", Epoches)

  LSTM_mod = Sequential()
  LSTM_mod.add(LSTM(200, input_shape=(X_trn.shape[1], 1)))
  LSTM_mod.add(Dense(1, activation='sigmoid'))
  LSTM_mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  LSTM_mod.fit(X_trn, y_trn, epochs=Epoches, batch_size=128)

  Y_pred = LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the count of epoches  5
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.5569 - loss: 0.6801
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.5944 - loss: 0.6628
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.6328 - loss: 0.6360
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.6657 - loss: 0.6072
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.6809 - loss: 0.5977
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


 Validation Score: 0.6835
              precision    recall  f1-score   support

           0       0.70      0.77      0.73      3015
           1       0.66      0.57      0.61      2305

    accuracy                           0.68      5320
   macro avg       0.68      0.67      0.67      5320
weighted avg       0.68      0.68      0.68      5320

Macro_F1_score 0.6712304315449213
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step


 Testing Score: 0.6808
              precision 

BiLSTM

In [83]:
X_trn = np.expand_dims(X_trn, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_tst = np.expand_dims(X_tst, axis=-1)

In [84]:
# hyper tuning the optimizer
for optimizer in ['adam', 'adamax', 'adagrad']:
  print("Result of hyper tuning the optimizer ", optimizer)

  bi_LSTM_mod = Sequential()
  bi_LSTM_mod.add(Bidirectional(LSTM(200, input_shape=(X_trn.shape[1], 1))))
  bi_LSTM_mod.add(Dense(1, activation='sigmoid'))
  bi_LSTM_mod.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  bi_LSTM_mod.fit(X_trn, y_trn, epochs=20, batch_size=128)

  Y_pred = bi_LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = bi_LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the optimizer  adam
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 89ms/step - accuracy: 0.5718 - loss: 0.6705
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.6745 - loss: 0.6039
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 86ms/step - accuracy: 0.6876 - loss: 0.5877
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 87ms/step - accuracy: 0.7015 - loss: 0.5732
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 87ms/step - accuracy: 0.7069 - loss: 0.5682
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 86ms/step - accuracy: 0.7106 - loss: 0.5650
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 87ms/step - accuracy: 0.7213 - loss: 0.5560
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 86ms/step - accuracy: 0.7144 - loss: 0.5623
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.7200 - loss: 0.5556
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 82ms/step - accuracy: 0.7102 - loss: 0.5619
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 89ms/step - accuracy: 0.7243 - 

In [85]:
# hyper tuning the count of epoches
for Epoches in [5, 10, 20]:
  print("Result of hyper tuning the count of epoches ", Epoches)

  bi_LSTM_mod = Sequential()
  bi_LSTM_mod.add(Bidirectional(LSTM(200, input_shape=(X_trn.shape[1], 1))))
  bi_LSTM_mod.add(Dense(1, activation='sigmoid'))
  bi_LSTM_mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  bi_LSTM_mod.fit(X_trn, y_trn, epochs=Epoches, batch_size=128)

  Y_pred = bi_LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = bi_LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the count of epoches  5
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step - accuracy: 0.5699 - loss: 0.6702
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - accuracy: 0.6767 - loss: 0.6040
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 85ms/step - accuracy: 0.6949 - loss: 0.5785
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 85ms/step - accuracy: 0.7093 - loss: 0.5690
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - accuracy: 0.7098 - loss: 0.5687
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step


 Validation Score: 0.7002
              precision    recall  f1-score   support

           0       0.71      0.81      0.75      3015
           1       0.69      0.56      0.62      2305

    accuracy                           0.70      5320
   macro avg       0.70      0.68      0.69      5320
weighted avg       0.70      0.70      0.69      5320

Macro_F1_score 0.6854112322952987
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


 Testing Score: 0.7032
              precis